In [ ]:
import numpy as np

import msq_field2csv as mq
import create_report as cr

import mosaiq_field_export as export
import mosaiq_connection as connect

In [ ]:
patient_ids = {
    'nbccc': '000000',
    'rccc': '999998'
}

sql_users = {
    'nbccc': 'physics',
    'rccc': 'physics'
}

sql_servers = {
    'nbccc': 'nbccc-msq',
    'rccc': 'msqsql'
}

qa_centre = 'rccc'
clinical_centre = 'nbccc'

qa_site_name = patient_ids[clinical_centre]
# qa_site_name = 'BLADDER Fx6to25'

In [ ]:
qa_user = sql_users[qa_centre]
qa_server = sql_servers[qa_centre]
qa_id = patient_ids[qa_centre]

patient_id = patient_ids[clinical_centre]

with connect.mosaiq_connect(qa_user, qa_server) as cursor:
    qa_fields = np.array(export.patient_fields(cursor, qa_id))

site_name = qa_fields[:,5]
field_ids = qa_fields[:,0]
mu = qa_fields[:,4]
reference = site_name == qa_site_name

qa_field_ids = field_ids[reference]
total_qa_mu = np.sum(mu[reference])
total_qa_mu

In [ ]:
clinical_user = sql_users[clinical_centre]
clinical_server = sql_servers[clinical_centre]

with connect.mosaiq_connect(clinical_user, clinical_server) as cursor:
    fields = np.array(export.patient_fields(cursor, patient_id))
    
valid_field = fields[:,3] == 0
has_mu = fields[:,4] != 0
clinical_fields = fields[valid_field & has_mu]

total_clinical_mu = np.sum(clinical_fields[:,4])
mu_diff = total_clinical_mu - total_qa_mu

clinical_field_ids = clinical_fields[:,0]

print(total_clinical_mu)
# clinical_fields

In [ ]:
clinical_field_ids

In [ ]:
fields = {
    'nbccc': [clinical_field_ids[0]],
    'rccc': qa_field_ids
}

mq.display_fields_overview(fields, sql_users, sql_servers)

In [ ]:
input_directory = r'S:\IMRT_QA\RCCC_to_NBCCC_Transfer_Checks\mosaiq_exports\{}'.format(patient_id)

mq.save_data_to_csv(input_directory, fields, sql_users, sql_servers)

In [ ]:
output_directory = r'S:\IMRT_QA\RCCC_to_NBCCC_Transfer_Checks\reports'

cr.create_reports(input_directory, output_directory)

In [ ]:
# from mosaiq_sql.execution import _get_raw_field_item
# from mosaiq_sql import sql_connection

# from IPython.display import display

# SQL_USERS = {
#     'nbccc': 'physics',
#     'rccc': 'physics'
# }
# SQL_SERVERS = {
#     'nbccc': 'physsvr',
#     'rccc': 'msqsql'
# }

In [ ]:
# field_id = 0
# field_label = 'Version'

# with sql_connection(SQL_USERS, SQL_SERVERS) as cursors:
#     cursors['nbccc']
#     display(_get_raw_field_item(cursors['nbccc'], field_label, field_id))